<a href="https://colab.research.google.com/github/roywaswa/gee_learner/blob/main/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import folium

In [ ]:
ee.Authenticate()
ee.Initialize()

In [4]:
landcover = ee.ImageCollection('MODIS/006/MCD12Q1')
land_surface_temp = ee.ImageCollection('MODIS/006/MOD11A1')
elevation = ee.Image('USGS/SRTMGL1_003')